In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import BytesIO
from zipfile import ZipFile
import requests
from datetime import datetime
import pytz
from tzwhere import tzwhere

In [2]:
#! ls

In [3]:
sightings = pd.read_pickle('sightings_w_location.pkl')

In [4]:
sightings

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State,Location,Lat,Lon
0,4/23/21 06:30,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting,NaN,United States,Blackshear,GA,"Blackshear, GA United States",31.30443,-82.240580
1,4/23/21 06:00,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...,NaN,United States,Thompson,VA,"Thompson, VA United States",37.78198,-79.974385
2,4/23/21 06:00,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...,NaN,United States,Vero Beach,FL,"Vero Beach, FL United States",27.63885,-80.393960
3,4/23/21 05:59,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...,NaN,United States,Saint Augustine,FL,"Saint Augustine, FL United States",29.89469,-81.314520
4,4/23/21 05:58,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...,NaN,United States,Durham,NC,"Durham, NC United States",35.99542,-78.896440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97243,,Disk,2 seconds,UFO disappears over cirrus formation.,1/11/19,http://www.nuforc.org/webreports/144/S144329.html,\nUFO disappears over cirrus formation.This mo...,ufo disappears cirrus formation.this morning a...,NaN,United States,Littleton,CO,"Littleton, CO United States",39.61691,-105.015050
97244,,Disk,1 minute,Spinning craft ( counter clockwise) I have video,12/23/20,http://www.nuforc.org/webreports/160/S160854.html,\nSpinning craft ( counter clockwise) I have v...,spinning craft ( counter clockwise ) videoi sh...,NaN,United States,Carson City,NV,"Carson City, NV United States",39.16506,-119.766970
97247,,Sphere,,A hovering sphere shaped object that appeared ...,4/8/19,http://www.nuforc.org/webreports/145/S145475.html,\nA hovering sphere shaped object that appeare...,hovering sphere shaped object appeared spinnin...,NaN,United States,Gettysburg,PA,"Gettysburg, PA United States",39.83025,-77.231120
97248,,Light,5 minutes,Blinking lights weaving between trees in forest.,4/8/19,http://www.nuforc.org/webreports/145/S145479.html,\nBlinking lights weaving between trees in for...,blinking light weaving tree forest.on back dec...,NaN,United States,Ricetown,KY,"Ricetown, KY United States",37.39224,-83.624110


In [5]:
# making a test subset for developing functions
#test = sightings.head(1000).copy()
sightings.dtypes

Date_Time               object
Shape                   object
Duration                object
Summary                 object
Posted                  object
Detail_Link             object
Detail_Summary          object
Detail_Summary_nltk     object
Notes                   object
Country                 object
City                    object
State                   object
Location                object
Lat                    float64
Lon                    float64
dtype: object

In [6]:
#test['Date'] = test['Date_Time'].str.split(" ").str[0].str.split('/').apply(lambda x: [str(val).zfill(2) for val in x]).str.join('/')
#test['Date']

In [7]:
#test['Time'] = test['Date_Time'].str.split(" ").str[1].str.split('/').str.join("")
#test['Time']

In [8]:
#test['Date_Time_type'] = test['Date'] + " "+ test['Time']
#test['Date_Time_type']

In [9]:
sightings['Date_Time'] = pd.to_datetime(sightings['Date_Time'], format = '%m/%d/%y %H:%M', errors='coerce')

In [10]:
sightings['Date_Time'] 

0       2021-04-23 06:30:00
1       2021-04-23 06:00:00
2       2021-04-23 06:00:00
3       2021-04-23 05:59:00
4       2021-04-23 05:58:00
                ...        
97243                   NaT
97244                   NaT
97247                   NaT
97248                   NaT
97249                   NaT
Name: Date_Time, Length: 61647, dtype: datetime64[ns]

In [11]:
tzwhere = tzwhere.tzwhere()

/Users/daisy/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [12]:
sightings['timezone_str']  = sightings.apply(lambda x: tzwhere.tzNameAt(x.Lat, x.Lon), axis = 1)

In [13]:
sightings['timezone_str']

0           America/New_York
1           America/New_York
2           America/New_York
3           America/New_York
4           America/New_York
                ...         
97243         America/Denver
97244    America/Los_Angeles
97247       America/New_York
97248       America/New_York
97249       America/New_York
Name: timezone_str, Length: 61647, dtype: object

In [14]:
#tzwhere = tzwhere.tzwhere()
#timezone_str = tzwhere.tzNameAt(31.3060513, -82.2420661) # Blackshear, GA
#timezone_str


In [15]:
Time_UTC = []
for t,dt in zip(sightings.timezone_str, sightings.Date_Time): 
    try:
        local = pytz.timezone(t)
        naive = datetime.strptime(str(dt), "%Y-%m-%d %H:%M:%S")
        local_dt = local.localize(naive, is_dst=None)
        utc_dt = local_dt.astimezone(pytz.utc)
        Time_UTC.append(utc_dt)
    except:
        Time_UTC.append(None)

In [16]:
sightings['Time_UTC'] = Time_UTC
sightings['Time_UTC']

0       2021-04-23 10:30:00+00:00
1       2021-04-23 10:00:00+00:00
2       2021-04-23 10:00:00+00:00
3       2021-04-23 09:59:00+00:00
4       2021-04-23 09:58:00+00:00
                   ...           
97243                         NaT
97244                         NaT
97247                         NaT
97248                         NaT
97249                         NaT
Name: Time_UTC, Length: 61647, dtype: datetime64[ns, UTC]

In [17]:
#timezone = pytz.timezone(timezone_str)
#dt = test['Date_Time_type'][0]
#local_dt = timezone.localize(dt, is_dst = None)
#
#dt_utc

In [18]:
#local = pytz.timezone(test['timezone_str'][0])
#naive = datetime.strptime(str(test['Date_Time'][0]), "%Y-%m-%d %H:%M:%S")
#local_dt = local.localize(naive, is_dst=None)
#utc_dt = local_dt.astimezone(pytz.utc)
#utc_dt

In [19]:
sightings.sample(8)

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State,Location,Lat,Lon,timezone_str,Time_UTC
48753,2013-07-25 06:00:00,Light,5 seconds,Bright orb like light shooting in multiple dir...,8/30/13,http://www.nuforc.org/webreports/100/S100456.html,\nBright orb like light shooting in multiple d...,bright orb like light shooting multiple direct...,NaN,United States,Fredonia,NY,"Fredonia, NY United States",42.44029,-79.33186,America/New_York,2013-07-25 10:00:00+00:00
6633,2020-03-10 21:10:00,Other,40 minutes,UFO Chase with my Shar Pei “NOVA.”,5/7/20,http://www.nuforc.org/webreports/154/S154160.html,\nUFO Chase with my Shar Pei “NOVA”On Tuesday ...,ufo chase shar pei “ nova ” tuesday 3-10-2020 ...,NaN,United States,West Helena,AR,"West Helena, AR United States",34.54409,-90.64443,America/Chicago,2020-03-11 02:10:00+00:00
85594,2007-01-27 23:07:00,Disk,"10,000",It was very bright and had lights that were ve...,2/1/07,http://www.nuforc.org/webreports/055/S55007.html,\nIt was very bright and had lights that were ...,"bright light red , disk silver , stayed air 5 ...",NaN,United States,Louisville,AK,"Louisville, AK United States",38.25489,-85.76666,America/New_York,2007-01-28 04:07:00+00:00
4040,2020-07-05 02:39:00,Light,3-5 minutes,Object that was orange circler bright light.,7/9/20,http://www.nuforc.org/webreports/157/S157346.html,\nObject that was orange circler bright lightM...,object orange circler bright lightmyself two p...,NaN,United States,Girardville,PA,"Girardville, PA United States",40.79163,-76.28165,America/New_York,2020-07-05 06:39:00+00:00
46523,2013-10-12 22:26:00,Circle,15 minutes,We just witnessed an amazing event. About ten...,10/14/13,http://www.nuforc.org/webreports/103/S103181.html,\nWe just witnessed an amazing event. About t...,witnessed amazing event . ten different hoveri...,NaN,United States,Hiawassee,GA,"Hiawassee, GA United States",34.95002,-83.75839,America/New_York,2013-10-13 02:26:00+00:00
44128,2014-01-13 18:45:00,Teardrop,3 minutes,"Silent, bright white falling comet like object...",1/16/14,http://www.nuforc.org/webreports/106/S106278.html,"\nSilent, bright white falling comet like obje...","silent , bright white falling comet like objec...",NaN,United States,Greensboro,NC,"Greensboro, NC United States",36.06908,-79.79503,America/New_York,2014-01-13 23:45:00+00:00
86880,2006-10-25 19:30:00,Circle,5 minutes,10-25-06 Circle North Dakota 5 minutes Shi...,12/7/06,http://www.nuforc.org/webreports/053/S53391.html,\n10-25-06 Circle North Dakota 5 minutes S...,10-25-06 circle north dakota 5 minute shiny ob...,NaN,United States,Mandan,ND,"Mandan, ND United States",46.82654,-100.88895,America/North_Dakota/New_Salem,2006-10-26 00:30:00+00:00
23860,2016-11-04 04:00:00,Formation,2 hours,((HOAX??)) I woke up around 4:00am on 11/4/16...,11/11/16,http://www.nuforc.org/webreports/130/S130996.html,\nI woke up to get a drink around 4:00am on 11...,woke get drink around 4:00am 11/4/16 ciggarite...,NaN,United States,Billings,MT,"Billings, MT United States",45.78426,-108.50607,America/Denver,2016-11-04 10:00:00+00:00
